In [1]:
import pandas as pd
import numpy as np
import torch
import csv
from sentence_transformers import SentenceTransformer, util
import torch.nn as nn
import torch.nn.functional as func
from torch.nn import Linear as lin
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from torch.distributions.multinomial import Multinomial
from transformers import BertForTokenClassification
from transformers import PreTrainedTokenizer
from transformers import AutoTokenizer
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForTokenClassification
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForMaskedLM
import random

/home/henry/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_test = pd.read_csv("Listing_Titles.tsv.gz", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)

In [3]:
def get_text(df):
    text = []
    for idx, row in df.iterrows():
        text.append(row["Title"].lower().split(" "))
        if idx%100000 == 0:
            print(idx)
    return text

In [4]:
train, test = train_test_split(get_text(df_test[:500000]), train_size=0.995)

0
100000
200000
300000
400000


In [5]:
for i in range(len(train)):
    for j in range(len(train[i])):
        r = random.random()
        if r <= 0.15:
            train[i][j] = "[MASK]"

for i in range(len(test)):
    for j in range(len(test[i])):
        r = random.random()
        if r <= 0.15:
            test[i][j] = "[MASK]"


In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(test, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [7]:
def encode_tags(labels, encodings):
    
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = np.array(doc_labels)[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)]


        encoded_labels.append(doc_enc_labels.tolist())
    

    return encoded_labels

t = encode_tags(train_encodings["input_ids"], train_encodings)
v = encode_tags(val_encodings["input_ids"], val_encodings)

In [8]:
train_tags = torch.where(torch.tensor(train_encodings.input_ids) == tokenizer.mask_token_id, torch.tensor(t), -100)
val_tags = torch.where(torch.tensor(val_encodings.input_ids) == tokenizer.mask_token_id, torch.tensor(v), -100)
train_encodings.pop("offset_mapping")
val_encodings.pop("offset_mapping")

[[(0, 0),
  (0, 6),
  (0, 5),
  (0, 8),
  (0, 5),
  (0, 6),
  (0, 5),
  (0, 7),
  (0, 2),
  (2, 6),
  (0, 6),
  (0, 4),
  (4, 7),
  (0, 4),
  (4, 7),
  (0, 4),
  (0, 3),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0)],
 [(0, 0),
  (0, 7),
  (0, 5),
  (0, 3),
  (3, 6),
  (0, 5),
  (0, 8),
  (0, 3),
  (0, 1),
  (0, 1),
  (0, 6),
  (0, 5),
  (0, 5),
  (0, 1),
  (0, 5),
  (0, 1),
  (0, 4),
  (4, 5),
  (5, 9),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0)],
 [(0, 0),
  (0, 7),
  (0, 6),
  (0, 6),
  (0, 5),
  (0, 6),
  (0, 5),
  (0, 4),
  (4, 7),
  (0, 1),
  (0, 5),
  (0, 6),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0)],
 [(0, 0),
  (0, 4),
  (0, 2),
  (2, 5

In [9]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        for encoding in self.encodings.keys():
            self.encodings[encoding] = torch.tensor(self.encodings[encoding]).to("cuda")
        self.labels = torch.tensor(labels).to("cuda")
        

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
train_dataset = Data(train_encodings, train_tags)
val_dataset = Data(val_encodings, val_tags)

/tmp/ipykernel_27587/2643714077.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels).to("cuda")


In [11]:
torch.cuda.set_per_process_memory_fraction(1.0, device=None)

In [15]:
torch.save(model.state_dict(), "/home/henry/work/projects/bert_1_epoch.obj")

In [13]:
#m = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
#m.load_state_dict(torch.load("/home/henry/work/projects/bert_1_epoch.obj"))

In [14]:
from transformers import DistilBertForMaskedLM, Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="./results", learning_rate=0.0001, num_train_epochs=1,per_device_train_batch_size=32,per_device_eval_batch_size=64,evaluation_strategy="steps", do_eval=True, eval_steps=250, warmup_steps=500,weight_decay=0.01,logging_dir='./logs',logging_steps=10,dataloader_pin_memory=False)

model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased").to("cuda")

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset             
)

trainer.train()

/home/henry/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 497500
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 15547
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hwarzecha (uchcia). Use `wandb login --relogin` to force relogin


  0%|          | 0/15547 [00:00<?, ?it/s]/tmp/ipykernel_27587/2643714077.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_27587/2643714077.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
  0%|          | 11/15547 [00:02<52:27,  4.94it/s] 

{'loss': 14.0513, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  0%|          | 21/15547 [00:04<51:35,  5.02it/s]

{'loss': 13.0515, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}


...

100%|█████████▉| 15541/15547 [55:26<00:01,  4.99it/s]

{'loss': 0.0, 'learning_rate': 4.652090117631422e-08, 'epoch': 1.0}


100%|██████████| 15547/15547 [55:27<00:00,  5.06it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 15547/15547 [55:27<00:00,  4.67it/s]

{'train_runtime': 3328.8002, 'train_samples_per_second': 149.453, 'train_steps_per_second': 4.67, 'train_loss': 0.03539331228952624, 'epoch': 1.0}


TrainOutput(global_step=15547, training_loss=0.03539331228952624, metrics={'train_runtime': 3328.8002, 'train_samples_per_second': 149.453, 'train_steps_per_second': 4.67, 'train_loss': 0.03539331228952624, 'epoch': 1.0})